<a href="https://colab.research.google.com/github/trashchenkov/LLM-First-Touch/blob/main/GigaChat3_10B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка и запуск модели GigaChat3-10B

В бесплатном Colab доступна GPU NVIDIA T4 (15 GB VRAM). Полная 10B-модель в “чистом” виде на такую карту не помещается, поэтому мы:

- Берём bf16-чекпойнт модели.

- Загружаем его и квантуем в 4 бита с помощью bitsandbytes.

Так мы получаем компактную версию, которая реально работает на T4.

In [1]:
!pip install -U accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 15.7 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0


## Почему берём bf16-чекпойнт и что такое bf16

На HuggingFace у этой модели есть разные варианты:

- ai-sage/GigaChat3-10B-A1.8B — FP8-квантованный чекпойнт. Он компактный и быстрый,
но требует GPU с compute capability ≥ 8.9 (например 4090/H100).
- T4 имеет capability 7.5 → FP8 на нём не поддерживается, поэтому запустить такую версию не получится.

- ai-sage/GigaChat3-10B-A1.8B-bf16 — исходные веса в формате bfloat16.

bf16 (bfloat16) — это 16-битный формат чисел с плавающей точкой:

- по размеру как fp16 (16 бит),

- но с большим диапазоном значений (как у fp32),

- поэтому его часто используют для хранения/обучения LLM.

Важно:
сам T4 плохо поддерживает вычисления в bf16, но мы берём bf16-чекпойнт как исходник,
а затем всё равно загружаем и считаем в fp16/4-битах. То есть bf16 тут — про формат весов на диске, а не про то, что модель прямо будет считать в bf16 на T4.

In [2]:
import torch

print("GPU:", torch.cuda.get_device_name(0))
print("Compute capability:", torch.cuda.get_device_capability(0))
print("VRAM (GB):", torch.cuda.get_device_properties(0).total_memory / 1024**3)


GPU: Tesla T4
Compute capability: (7, 5)
VRAM (GB): 14.74127197265625


## Откуда берётся 4-битная версия

4-битная модель не лежит готовой в репозитории.
Мы делаем её сами при загрузке:

Скачиваются полные bf16-шарды весов (они большие — это нормально).

Библиотека bitsandbytes перехватывает загрузку и:

- хранит веса в 4-битном виде (NF4),

- а при вычислениях раскладывает их во внутренний fp16-буфер.

Это называется on-the-fly quantization — квантование “на лету”.
Именно поэтому размер скачивания большой, но в VRAM модель занимает уже существенно меньше.

In [3]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    GenerationConfig,
)

model_name = "ai-sage/GigaChat3-10B-A1.8B-bf16"

# Настройка 4-битной квантовки для T4
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,  # на T4 надежнее fp16
)

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    quantization_config=quant_config,
    dtype=torch.float16,
    trust_remote_code=True,  # у модели кастомные модули DeepSeekV3/MLA/MoE
)

model.generation_config = GenerationConfig.from_pretrained(model_name)

print("Model loaded. Device map:")
print(model.hf_device_map)


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/10.7M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/276 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

model-00008-of-00010.safetensors:   0%|          | 0.00/4.08G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/11 [00:00<?, ?it/s]

Some weights of the model checkpoint at ai-sage/GigaChat3-10B-A1.8B-bf16 were not used when initializing DeepseekV3ForCausalLM: ['model.layers.26.eh_proj.weight', 'model.layers.26.embed_tokens.weight', 'model.layers.26.enorm.weight', 'model.layers.26.hnorm.weight', 'model.layers.26.input_layernorm.weight', 'model.layers.26.mlp.experts.0.down_proj.weight', 'model.layers.26.mlp.experts.0.gate_proj.weight', 'model.layers.26.mlp.experts.0.up_proj.weight', 'model.layers.26.mlp.experts.1.down_proj.weight', 'model.layers.26.mlp.experts.1.gate_proj.weight', 'model.layers.26.mlp.experts.1.up_proj.weight', 'model.layers.26.mlp.experts.10.down_proj.weight', 'model.layers.26.mlp.experts.10.gate_proj.weight', 'model.layers.26.mlp.experts.10.up_proj.weight', 'model.layers.26.mlp.experts.11.down_proj.weight', 'model.layers.26.mlp.experts.11.gate_proj.weight', 'model.layers.26.mlp.experts.11.up_proj.weight', 'model.layers.26.mlp.experts.12.down_proj.weight', 'model.layers.26.mlp.experts.12.gate_proj.w

generation_config.json:   0%|          | 0.00/90.0 [00:00<?, ?B/s]

Model loaded. Device map:
{'': 0}


## Чат-шаблон и корректная остановка генерации

У модели есть chat_template (файл `chat_template.jinja` в репозитории).
Метод `tokenizer.apply_chat_template(...)` превращает список сообщений в правильный промпт.

Важно правильно остановить генерацию:

мы явно передаём `eos_token_id` и `pad_token_id`,

и ставим разумный `max_new_tokens`, чтобы модель не “уплывала” в другой стиль текста.

In [4]:
messages = [
    {"role": "user", "content": "Что ты знаешь о конференции AI Journey?"}
]

input_tensor = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

gen_cfg = model.generation_config

# eos у некоторых моделей может быть списком
eos_ids = gen_cfg.eos_token_id or tokenizer.eos_token_id
pad_id = eos_ids[0] if isinstance(eos_ids, list) else eos_ids
pad_id = pad_id or tokenizer.eos_token_id

with torch.no_grad():
    outputs = model.generate(
        input_tensor,
        max_new_tokens=300,
        do_sample=False,
        temperature=0.0,
        eos_token_id=eos_ids,
        pad_token_id=pad_id,
    )

result = tokenizer.decode(
    outputs[0][input_tensor.shape[1]:],
    skip_special_tokens=True
)

print(result)


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Конференция AI Journey — это одно из ключевых мероприятий в области искусственного интеллекта и машинного обучения, организуемое Сбером. Она собирает экспертов, исследователей, разработчиков и энтузиастов для обсуждения последних достижений, тенденций и перспектив развития технологий ИИ.

Основные темы и направления, которые обычно затрагиваются на конференции AI Journey, включают:

1. **Технологии искусственного интеллекта**: Обсуждение новых алгоритмов, моделей машинного обучения, нейронных сетей и других методов, используемых в ИИ.
2. **Практическое применение ИИ**: Применение искусственного интеллекта в различных сферах, таких как здравоохранение, финансы, образование, промышленность и другие.
3. **Этические аспекты ИИ**: Рассмотрение вопросов этики, безопасности и ответственности при разработке и использовании систем искусственного интеллекта.
4. **Будущее ИИ**: Прогнозирование будущих трендов и возможностей в развитии искусственного интеллекта.
5. **Инновации и стартапы**: Обмен 